In [36]:
# Stylianos Topalidis
# AEM: 9613
# email: styltopa@ece.auth.gr 

In [37]:
from sklearn import datasets, tree
from pprint import pprint
import numpy as np


In [38]:
iris = datasets.load_iris()
# print(dir(iris))

# First attribute: sepal length (cm)
# Second attribute: sepal width (cm)
irisData = iris.data[:, 0:2]


# Fields of iris data object
# ['DESCR', 'data', 'data_module', 'feature_names', \
#  'filename', 'frame', 'target', 'target_names']

# Training percent
trainPercent = 0.5
# testPercent = 1 - trainPercent
# print(type(iris.data))

# Turn the target values into a set of all possible targets {0, 1, 2} 
# to exclude duplicates
singleTargetValsSet = set(iris.target)

# Turn the set into a list and then into a numpy array
# for convenience 
singleTargetValsList = list(singleTargetValsSet)
singleTargetValsArr = np.array(singleTargetValsList) 


# Indexes for all the data samples (iterates over the data 
# for the different targets: setosa, versicolor, virginica) 
indexesListPerIdentifier = []

# Indexes for the training and testing data in the original dataset
trainingIndexesList = []
testingIndexesList = []

# for each of the flower types
for targetIdentifier in range(len(singleTargetValsArr)):
    indexesArr = np.where(iris.target == targetIdentifier)

    # the first element of the list are the actual data 
    # and the second one is its data type 
    indexesListPerIdentifier = list(indexesArr[0])

    # number of training data derived from the training percentage selected
    # per group identifier (0, 1, 2)
    numOfTrainingData = round(trainPercent*len(indexesListPerIdentifier))
    numOfTestingData = len(indexesListPerIdentifier) - numOfTrainingData

    # concatenate the indices of the new target (identifier) training data 
    # with the indices of the old target training data.
    trainingIndexesList = trainingIndexesList + indexesListPerIdentifier[0:numOfTrainingData]
    testingIndexesList = testingIndexesList + indexesListPerIdentifier[numOfTrainingData:]
    


# print(trainingIndexesList)
# print(testingIndexesList)


In [39]:
# Arrays holding the indexes of the training and testing data samples
# list -> np.array
trainingIndexesArr = np.array(trainingIndexesList)
testingIndexesArr = np.array(testingIndexesList)

# To avoid training the tree with batches of the same target data,
# as given (all setosa first, then all versicolor and finally all virginica),
# we permute the training data
np.random.seed(0)
trainingIndexesArrPerm =  np.random.permutation(trainingIndexesArr)
trainingArrPerm = irisData[trainingIndexesArr]



In [47]:
# Training data samples
trainingArr = trainingArrPerm

# Target values of the training data samples
targetArrTraining = iris.target[trainingIndexesArrPerm]


# print(targetArr)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(trainingArr, targetArrTraining)


testingArr = irisData[testingIndexesArr]
targetArrTesting = iris.target[testingIndexesArr]


samPred = clf.predict(testingArr)

# # correpredictions
targetAndPred = np.stack((targetArrTesting, samPred), axis=1) 

correctlyPredicted = 0

for predCount in range(len(samPred)):
    # print(targetAndPred[predCount])
    if iris.target[predCount] == samPred[predCount]:
        correctlyPredicted = correctlyPredicted + 1
    # else:
        # print('Mispredicted')



accuracy = correctlyPredicted/numOfTestingData 
print('Accuracy: ', accuracy*100, '%')

Accuracy:  92.0 %
